In [73]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### 1. Instructions:

In order to run the code properly, please follow this recommended procedure:

1)	Click on "Cell" and then click on "Run All".

2)	Select 'Test Type' parameter (i.e., Cyclic or Monotonic). The Available Test Cases corresponding to this Test Type will be shown on the DropDown Manu below. The description details of these available test cases will be shown in the text area to the right of the test case selection area. 

3) Select 'Stage' parameter (i.e., Consolidation or Shear). 

4) Select 'Test Case(s)' and start plotting. Click on a particular test case (e.g., 'SW1') to start plotting. To plot **multiple** Test Cases simultaneously, strike **'Ctrl'** key (or **'Command'** key for Mac) and hold, then start selecting multiple test cases. As one add or drop test cases, the plot area will be dynamically updated. 

Note: DSS = 'Direct Simple Shear'

In [74]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [75]:
%matplotlib notebook
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib import rc
from matplotlib import ticker
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
from ipywidgets import *
from IPython.display import display
from IPython.display import clear_output
from scipy.signal import find_peaks_cwt
from scipy import optimize
from scipy.optimize import leastsq
import statsmodels.api as sm
import os

font = {'color':  'black',
        'weight': 'normal',
        'size': 12,
        'style': 'italic'
        }

matplotlib.rcParams.update({'font.family': 'sans-serif','font.sans-serif': 'Arial', 'mathtext.fontset': 'stix',
                           'font.size':'12'})
matplotlib.rcParams['axes.linewidth'] = 1.5
formatter = ticker.ScalarFormatter(useMathText=True)

In [91]:
# Init Widget Elements 
# if new mix types are added, the 'mix_type' list should be updated, as well as on_options_change function at the end.
mix_type = ['SBFW','SKFW','SBSW']

select_mix_type = widgets.RadioButtons(options=mix_type, 
                                       description='Soil Mixture Type (Event)', 
                                       variable=[True, False],
                                       layout = Layout(width='25%')) 

radio_btn_test_type = widgets.RadioButtons(options=['Monotonic','Cyclic'],
                                           description='DSS Test Type',
                                           variable=[True,False],
                                           layout = Layout(width='25%'))

radio_btn_stage = widgets.RadioButtons(options=['Consolidation','Shear'],
                                       description='Test Stage',
                                       variable=[True,False],
                                       layout = Layout(width='25%'))

slider_param = FloatSlider(min=1, max=20, step=0.5, 
                           description = "Strain Level(%)", 
                           continuous_update = False,
                           layout = Layout(width='25%'))
slider_param.value = 3

layout_test = widgets.Layout(border = '0px', 
                             width='100%', 
                             height='100%',
                             flex_flow='column',
                             display='flex',
                             align_items='stretch')

files = os.listdir('./DSS_Dataset/')
test = [f.strip('.txt') for f in files if ('MON' in f) and ('_C.' in f) and f.startswith('SBSW')]

select_multi_test = widgets.SelectMultiple(description="Tests ID", 
                                           options= test, 
                                           disabled=False, 
                                           layout=layout_test) 

ta = Textarea(value= "", 
              disabled=True,
              layout=Layout(display='flex',flex_flow='row', border='solid 1px',
                            align_items='stretch',width='90%', height='100%'),
              description="Details of Test Cases")

btn_save_figure = widgets.Button(description='Save Figure')

In [77]:
###### Init input parameters #########
# All params defined here are global variables and are assumed to be visible throughout the notebook 
file_description = 'Description.csv'
#############################################################################################
info = pd.read_csv(file_description,delimiter=',',dtype='str',skiprows=13)
info = info.set_index('HEADING')
dssg = info.loc[:'GROUP']
dsst = info.loc['HEADING':]
dsst_col = dsst.loc['HEADING'].tolist()
dsst.columns = dsst_col
dsst = dsst[1:]
#############################################################################################

In [78]:
def on_button_save_fig_clicked(b):
    '''
    callback function for Saving Output Figures
    '''
#     pdf = matplotlib.backends.backend_pdf.PdfPages(text_fig_name.value)
    png = matplotlib.backends.backend_agg.Figure
    label = str(select_multi_test.value)
    
    if radio_btn_stage.value == "Consolidation":
        lab = label+'consolidation'
        png.savefig(b.fig_consol, lab, dpi=300)
    elif radio_btn_stage.value == "Shear": 
        lab1 = label+'shear1'
        lab2 = label+'shear2'
        png.savefig(b.fig_shear1, lab1, dpi=300)
        png.savefig(b.fig_shear2, lab2, dpi=300)
        
    png.clear()
    print('Figure Exported.')
    
# add action to be performed on click
btn_save_figure.on_click(on_button_save_fig_clicked)

In [79]:
def get_descriptions(dssg, test, test_type):

    if test_type == "Cyclic":
        descriptions = ""
    elif test_type == "Monotonic":
        descriptions = ""
        for test_case in test: 
            this_row = dssg[dssg['SPEC_REF']==test_case]

            if len(this_row) != 0:
                new_description_to_append =  test_case + ": " + \
                    str(this_row["DSSG_DESC"].values[0]) + "\n"
            else:
                new_description_to_append = test_case + ": None\n"
            descriptions = descriptions + new_description_to_append            
    return descriptions

############# get detailed descriptions ##################################
def describe(test_type, test_name, description_g, description_t, IVRt):
    if (description_t['DSST_IVS'].tolist() == []):
        print("No description yet.")
        return 

    if test_type == "Monotonic":
        print(test_name +' DESCRIPTION: \n'+ description_g['DSSG_DESC'].tolist()[0])
        print('And '+description_t['DSST_DESC'].tolist()[0])
        print('Plastic Limit(PL) = ' + description_g['DSSG_PL'].tolist()[0]+
            ';\nLiquid Limit(LL) = ' + description_g['DSSG_LL'].tolist()[0]+
            ';\nPlastic Index(PI) = ' + description_g['DSSG_PI'].tolist()[0]+
            ';\nMax. past pressure Sigma_p = '+ description_g['DSSG_SIGP'].tolist()[0]+'kPa;'
             )
        if IVRt == 0.9:
            print('Initial Void Ratio assumed 0.9;')
    else:
        print('No description yet.')
 

In [80]:
def fit_csl(sigma_v, e_c, axes):

#     x is in log(sigma_v), note: log is ln here
#     y is e_c
#     fit a linear regression line    
#     y =  Gamma + Lambda * ln(x)

    # transform to log(sigma_v)
    logx = np.log(sigma_v)
    X = sigma_v
    y = e_c #dependent variable
    
    # y = a  + b * x
    logx = sm.add_constant(logx)
    
    # construct a model, which is an ordinary least square model (OLS)
    model = sm.OLS(y,logx)
    results = model.fit()   
    
    Gamma = results.params[0]
    Lambda = results.params[1]
    
    
    x_new = np.linspace(np.min(logx), np.max(logx*1.2),100)
    y_new = Gamma + Lambda * x_new
    #axes.semilogx(x_new, powerlaw(x_new, a, b))
    e_logx = np.exp(x_new)

    axes.semilogx(e_logx, y_new)
    
    return (Gamma, Lambda, results.rsquared)

In [81]:
def fit_powerlaw(x, y, axes):

#     Given x and y, this function solves for S and m.

    yerr = [i * 0.2 for i in y]
    logx = np.log10(x)
    logy = np.log10(y)

    yerr = np.asarray(yerr)
    y = np.asarray(y)
    x = np.asarray(x)
    logyerr = yerr / y

    fitfunc = lambda p, x: p[0] + p[1] * x
    errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err

    pinit = [1.0, -1.0]
    out = optimize.leastsq(errfunc, pinit,
                           args=(logx, logy, logyerr), full_output=1)

    pfinal = out[0]
    covar = out[1]
    b = pfinal[1]
    a = 10.0**pfinal[0]

    powerlaw = lambda x, a, b: a * (x**b)
    
    x_new = np.linspace(np.min(x*0.8), np.max(x*1.2),100)
    axes.semilogx(x_new, powerlaw(x_new, a, b))
    
    return (a,b)

In [82]:
def plot_CSR(time, shear_strain, ax, peak_avg, period, initial_vertical_stress,label_CSR):
    
    time_def_percent = time[-1]
    CSR = round(float(peak_avg/initial_vertical_stress),4)
    Number_of_Cycle = round(float(time_def_percent/period),1)
                
    ax.semilogx(Number_of_Cycle, CSR, 'o', label = label_CSR)
    fmt = FuncFormatter(lambda x, pos: str(int(x)))
    ax.xaxis.set_major_formatter(fmt)
    ax.set_xlabel('Number of Uniform Cycle, N', fontdict=font)
    ax.set_ylabel('Cyclic Stress Ratio, $\\tau_{cyc}/\sigma\'_{vc}$', fontdict=font)
    ax.set_title('CSR vs. N', fontdict=font)
    ax.set_xlim(left=0.1)
    ax.grid(True, which='both')
    ax.legend(loc='best',prop={'size':10}) 
    ax.tick_params(direction='in',width=1.5,which='both')
    
    print (label_CSR + '; CSR ='+ str(CSR) + '; Number of Uniform Cycle: '+str(Number_of_Cycle))
    
    return (Number_of_Cycle, CSR)

In [83]:
def strength_normalize(OCR, initial_vertical_stress, shear_stress, ax, strain_limit, lab):

    ## Su is the maximum value of the shear stress
    ## sigma_vc is the value of the vertical stress at the beginning of test
    ## Su_sigma is a constant value for a test.

    residual = max(shear_stress)
    Su_sigma = residual/initial_vertical_stress

    ax.semilogx(OCR,Su_sigma,'o',label=lab)
    fmt = FuncFormatter(lambda x, pos: str(round(float(x),1)))
    fmt2 = FuncFormatter(lambda x, pos: str(int(x)))
    ax.xaxis.set_minor_formatter(fmt2)
    ax.xaxis.set_major_formatter(fmt)
    ax.set_title('Strength Normalization Plot')
    ax.set_ylabel(r'$\mathrm{\frac{S_u}{\sigma_v}}$', fontdict=font, rotation=360)
    ax.set_xlabel('Overconsolidation Ratio, OCR', fontdict=font)
    ax.legend(loc='best', prop={'size':10})
    ax.set_xlim([1,10])
    ax.grid(True, which="both")
    ax.tick_params(direction='in',width=1.5,which='both')
          
    return(OCR, Su_sigma)

In [84]:
def plot_all(stage, test_type, select_multi_test):
     
    test_chose = list(select_multi_test)
    L = len(test_chose)
    
    ###----for consolidation----###
    if stage == 'Consolidation':
        if L > 0:
            f, axes = plt.subplots(1,2,figsize=(8,4),dpi=80)
            btn_save_figure.fig_consol = f
            plt.rc('lines')
            
            for i in range(0,L):
                path = './DSS_Dataset/'+test_chose[i] + '.txt'
                data = np.genfromtxt(path, delimiter=',', usecols=np.arange(0,21))
                    
                if test_type != 'Cyclic': 
                    condition = ((dsst['SPEC_REF']==test_chose[i])&
                                (dsst['DSST_FSET']=='Consolidation.txt'))
                    description_t = dsst[condition]
                    description_g = dssg[(dssg['SPEC_REF']==test_chose[i])]
                    
                    # IVSt: Initial vertical stress
                    # FVSt: final vertical stress
                    # IVRt: initial void ratio
                    # FVRt: final void ratio
                    try:
                        IVSt = float(description_t['DSST_IVS'].tolist()[0])
                        FVSt = float(description_t['DSST_FVS'].tolist()[0])
                        IVRt = float(description_t['DSST_IVR'].tolist()[0])
                        FVRt = float(description_t['DSST_FVR'].tolist()[0])
                    except IndexError:
                        IVSt = 0
                        FVSt = 0
                        IVRt = 0.9

                    describe(test_type=test_type, test_name=test_chose[i], 
                             description_g=description_g, description_t=description_t, 
                             IVRt=IVRt)
                    
                time = data[2:, 0] - data[2,0]
                vertical_stress = data[2: ,18]
                pwp = data[2: ,20]
                axial_strain = data[2: ,15]
                shear_strain = data[2: ,14]
                max_vstress = max(vertical_stress)
                min_vstress = vertical_stress[-1] 
                OCR = str(int(round(max_vstress/min_vstress))) 
                lab = test_chose[i].strip('.txt')
                
                
                axes[0].plot(time, vertical_stress, linewidth=1, label=lab);
                axes[0].set_xlabel('Time (sec)',fontdict=font)
                axes[0].set_ylabel('Vertical Effective Stress, $\sigma\'_{vc}$ (kPa)',fontdict=font)
                axes[0].legend(loc='best',prop={'size':10})
                axes[0].grid(True)
                axes[0].tick_params(direction='in',width=1.5)
                f.tight_layout()

                axes[1].plot(time, axial_strain, linewidth=1, label=lab);
                axes[1].set_xlabel('Time (sec)', fontdict=font)
                axes[1].set_ylabel('Axial Strain, $\epsilon_a$ (%)', fontdict=font)
                axes[1].legend(loc='best',prop={'size':10})
                axes[1].grid(True)
                axes[1].tick_params(direction='in',width=1.5)
                f.tight_layout()               
                
    ###----for shear test----###        
    elif stage == 'Shear':
        if L > 0:
            f1, axes1 = plt.subplots(1,3,figsize=(12,4),dpi=80)
            f2, axes2 = plt.subplots(2,2,figsize=(10,8),dpi=80)
            btn_save_figure.fig_shear1 = f1
            btn_save_figure.fig_shear2 = f2
            plt.rc('lines')
            
            for i in range(0,L):
                
                path = './DSS_Dataset/' + test_chose[i] + '.txt'
                data = np.genfromtxt(path, delimiter=',')

                # reading data
                shear_strain_raw = data[2: ,14]
                strain_limit = slider_param.value
                if max(shear_strain_raw)<strain_limit:
                    is_abs_above_limit = abs(shear_strain_raw) == max(shear_strain_raw)
                    index_min2 = np.where(is_abs_above_limit)[0][0] + 1
                else:
                    is_abs_above_limit = abs(shear_strain_raw) > strain_limit
                    index_min2 = np.where(is_abs_above_limit)[0][0] + 1
                
                # different shear plots for cyclic (axes2[1,0] & axes2[1,1])
                if test_type == 'Cyclic':
                    lab = test_chose[i].strip('.txt')
                    driving = data[2: ,22]
                    start_index = np.where(driving>0)[0][0]+2
                    driving = driving[start_index:index_min2]
                    
                    pwp = data[start_index:index_min2,20]
                    vertical_stress = data[start_index:index_min2,18]
                    initial_vertical_stress = vertical_stress[0]
#                     Ru = pwp/initial_vertical_stress
                    
                    time = data[start_index:index_min2,0]-data[start_index,0]
                    shear_strain = data[start_index:index_min2,14]
                    shear_stress = data[start_index:index_min2,17]
                    normalized_shearstress = data[start_index:index_min2,-3]
                    normalized_verticalstress = data[start_index:index_min2,-2]
                    Ru = data[start_index:index_min2,-1]
                    void_ratio = 0*Ru
                    
                    
                    for n in range(1,len(driving)):
                        if (driving[n]>driving[n-1]) & (driving[n]>driving[n+1]):
                            first_peak=n
                            peak1=shear_stress[first_peak]
                            peak2=shear_stress[first_peak*3]
                            period = (time[n+1])*4
                            break
                    peak_avg =(abs(peak1)+abs(peak2))/2
                    cycle_all = time/period
                    
                    axes2[1,0].plot(cycle_all, shear_strain, linewidth=1, label=lab)
                    axes2[1,0].set_xlabel('Number of Uniform Cycles, N', fontdict=font)
                    axes2[1,0].set_ylabel('Shear Strain, $\gamma$ (%)', fontdict=font)
                    axes2[1,0].set_xlim(left=0)
                    axes2[1,0].grid(True)
                    axes2[1,0].tick_params(direction='in',width=1.5)
                    f2.tight_layout() 
                    
                    axes2[1,1].plot(cycle_all, Ru, linewidth=1, label=lab);
                    axes2[1,1].set_xlabel('Number of Uniform Cycles, N', fontdict=font)
                    axes2[1,1].set_ylabel('Normalized Pore Pressure Ratio, $r_u$', fontdict=font)
                    axes2[1,1].set_xlim(left=0)
                    axes2[1,1].grid(True)
                    axes2[1,1].tick_params(direction='in',width=1.5)
                    f2.tight_layout() 
                    
                # different shear plots for monotonic (axes2[1,0] & axes2[1,1])
                else:
                    shear_strain = data[2:index_min2,14]
                    time = data[2:index_min2,0] - data[2,0]
                    pwp = data[2:index_min2,20]
                    shear_stress = data[2:index_min2,17]
                    shear_stress_raw = data[2:,17]
                    vertical_stress = data[2:index_min2,18]
                    normalized_shearstress = data[2:index_min2,-3]
                    normalized_verticalstress = data[2:index_min2,-2]
                    Ru = data[2:index_min2,-1]
                    void_ratio = 0*Ru
                    initial_vertical_stress = vertical_stress[0]
                    
                    FVSt = vertical_stress[-1]
                    
                    initial_vertical_stress = vertical_stress[0] 
                    condition = (dsst['SPEC_REF']==test_chose[i])&(dsst['DSST_FSET']=='Shear.txt')
                    description_g = dssg[(dssg['SPEC_REF']==test_chose[i])]
                    description_t = dsst[condition]
                    
                    #initial and final void ratio:
                    try:
                        IVRt = float(description_t['DSST_IVR'].tolist()[0])
                        FVRt = float(description_t['DSST_FVR'].tolist()[0])
                        OCR = str(int(description_t['DSST_IOCR'].tolist()[0]))        
                        lab = test_chose[i].strip('.txt') +'; strain:'+ str(slider_param.value)+'%'
                        lab_normal = test_chose[i] + '; OCR = '+ OCR
                        
                    except IndexError:
                        IVRt = 0.9
                        FVRt = 0.8
                        vertical_stress_original = data[2: ,18]
                        max_vstress = max(vertical_stress_original)
                        min_vstress = vertical_stress_original[-1] 
                        OCR = str(int(round(max_vstress/min_vstress)))
                        lab = test_chose[i].strip('.txt')+'; strain:'+ str(slider_param.value)+'%'
                        lab_normal = test_chose[i] + '; OCR = '+ OCR
                        
                    
                    describe(test_type=test_type, test_name=test_chose[i], 
                             description_g=description_g, description_t=description_t, 
                             IVRt=IVRt)
                    
                    axes2[1,0].axis('off')
                    
                    axes2[1,1].plot(shear_strain, Ru, linewidth=1, label=lab)
                    axes2[1,1].set_xlim([0,20])
                    axes2[1,1].set_xlabel('Shear Strain, $\gamma$ (%)', fontdict=font)
                    axes2[1,1].set_ylabel('Normalized Pore Pressure Ratio, $r_u$ (kPa)', fontdict=font)
                    axes2[1,1].grid(True)
                    axes2[1,1].tick_params(direction='in',width=1.5)
                    f2.tight_layout() 
                    
                    pd.options.display.max_colwidth = 100

                # After getting time vectors for monotonic and cyclic respectively, plot time plots here
                # Time Plots for both monotonic and cyclic
                axes1[0].plot(time, shear_stress, linewidth=1, label=lab)
                axes1[0].set_xlabel('Time (sec)', fontdict=font)
                if test_type == 'Cyclic':
                    axes1[0].set_ylabel('Shear Stress, $\\tau_{cyc}$ (kPa)', fontdict=font)
                if test_type == 'Monotonic':
                    axes1[0].set_ylabel('Shear Stress, $\\tau_h$ (kPa)', fontdict=font)
                axes1[0].set_xlim(left=0)
                axes1[0].grid(True)
                axes1[0].tick_params(direction='in',width=1.5)
                f1.tight_layout()

                axes1[1].plot(time, shear_strain, linewidth=1, label=lab);
                axes1[1].set_xlabel('Time (sec)', fontdict=font)
                axes1[1].set_ylabel('Shear Strain, $\gamma$ (%)', fontdict=font)
                axes1[1].set_xlim(left=0)
                axes1[1].legend(loc='best',prop={'size':10})
                axes1[1].grid(True)
                axes1[1].tick_params(direction='in',width=1.5)
                f1.tight_layout()

                axes1[2].plot(time, pwp, linewidth=1, label=lab);
                axes1[2].set_xlabel('Time (sec)', fontdict=font)
                axes1[2].set_ylabel('Equivalent Pore Water Pressure, $u_e$ (kPa)', fontdict=font)
                axes1[2].set_xlim(left=0)
                axes1[2].grid(True)
                axes1[2].tick_params(direction='in',width=1.5)
                f1.tight_layout()
                
                # Shear Plots for both monotonic and cyclic
                # Normalized parameters to be used####################################To be changed
                axes2[0,0].plot(normalized_verticalstress, normalized_shearstress, 
                                linewidth=1, label=lab);
                axes2[0,0].set_xlabel('Normalized Vertical Stress, $\sigma_v\'/\sigma\'_{vc}$', fontdict=font)
                if test_type == 'Monotonic':
                    axes2[0,0].set_ylabel('Normalized Shear Stress, $\\tau_h/\sigma\'_{vc}$', fontdict=font)
                if test_type == 'Cyclic':
                    axes2[0,0].set_ylabel('Normalized Shear Stress, $\\tau_{cyc}/\sigma\'_{vc}$', fontdict=font)
                axes2[0,0].grid(True)
                axes2[0,0].tick_params(direction='in',width=1.5)
                f2.tight_layout()

                axes2[0,1].plot(shear_strain, normalized_shearstress, linewidth=1, label=lab);
                if test_type == 'Monotonic':
                    axes2[0,1].set_xlim([0,20])
                    axes2[0,1].set_ylabel('Normalized Shear Stress, $\\tau_h/\sigma\'_{vc}$', fontdict=font)
                if test_type == 'Cyclic':
                    axes2[0,1].set_ylabel('Normalized Shear Stress, $\\tau_{cyc}/\sigma\'_{vc}$', fontdict=font)
                axes2[0,1].legend(loc='best',prop={'size':10})
                axes2[0,1].set_xlabel('Shear Strain, $\gamma$ (%)', fontdict=font)
                
                axes2[0,1].grid(True)
                axes2[0,1].tick_params(direction='in',width=1.5)
                f2.tight_layout()
                

In [92]:
def on_options_change(*args):

    files = os.listdir('./DSS_Dataset/')
    test_type = radio_btn_test_type.value
    stage = radio_btn_stage.value
    mix_type = select_mix_type.value
    
    # if new mix types are added, should update accordingly
    if mix_type == 'SBSW':
        files = [f for f in files if f.startswith('SBSW')]
    if mix_type == 'SBFW':
        files = [f for f in files if f.startswith('SBFW')]
    if mix_type == 'SKFW':
        files = [f for f in files if f.startswith('SKFW')]   

    if test_type == 'Cyclic':
        test1 = [f for f in files if 'CYC' in f]
        if stage == 'Consolidation':
            test = [f.strip('.txt') for f in test1 if '_C.' in f]
        if stage == 'Shear':
            test = [f.strip('.txt') for f in test1 if '_S.' in f]
    if test_type == 'Monotonic':
        test1 = [f for f in files if 'MON' in f]  
        if stage == 'Consolidation':
            test = [f.strip('.txt') for f in test1 if '_C.' in f]
        if stage == 'Shear':
            test = [f.strip('.txt') for f in test1 if '_S.' in f]     
    
    select_multi_test.options = test
    
    ##### Update Text Area ########
    ta.value = get_descriptions(dssg = dssg, 
                                test = select_multi_test.options, 
                                test_type= radio_btn_test_type.value)
    
    
def update_plot(*args):
    clear_output()
    plot_all(stage = radio_btn_stage.value, test_type = radio_btn_test_type.value, 
            select_multi_test = select_multi_test.value)
    
def on_slider_change(*args):
    if (radio_btn_stage.value == "Shear") & (radio_btn_test_type.value == "Cyclic"):        
        update_plot(*args)
    if (radio_btn_stage.value == "Shear") & (radio_btn_test_type.value == "Monotonic"):  
        update_plot(*args)
        
# add test_type observer
radio_btn_test_type.observe(on_options_change, names="value")
radio_btn_stage.observe(on_options_change, names="value")
select_mix_type.observe(on_options_change, names="value")
select_multi_test.observe(update_plot, names="value")

# update plot when slider changes and filter is matched 
slider_param.observe(on_slider_change, names="value")

hbox_param_major = widgets.HBox([select_mix_type, radio_btn_test_type, radio_btn_stage, slider_param], 
                                layout = Layout(display='flex',
                                                flex_flow='row', 
                                                border='solid 1px',
                                                align_items='flex-start', 
                                                width='100%',
                                                height="100px"))

vbox_test = widgets.VBox([select_multi_test], 
                         layout = Layout(display='flex',
                                         flex_flow='row', 
                                         border='solid 1px',
                                         align_items='stretch', 
                                         width="400px",
                                         height='100%')) 

hbox_main =widgets.HBox([vbox_test, ta],
                        layout = Layout(display='flex',
                                        flex_flow='row',
                                        align_items='stretch', 
                                        width='100%',
                                        height="180px"))

hbox_save_fig = widgets.HBox([btn_save_figure],
                             layout = Layout(display='flex',
                                             flex_flow='row',
                                             align_items='center', 
                                             width='100%',
                                             height="50px"))                   


display(hbox_param_major)
display(hbox_main)
display(hbox_save_fig)